In [67]:

import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import tensorflow as tf


In [64]:

y = pd.read_csv('../../db/data/secondary/2010/logs_.csv', index_col=0)

y.reset_index(inplace=True, drop=True)

y = y[1:]

In [65]:
X = pd.read_csv('../../db/data/tertiary/2010/logs_.csv', index_col=0)

X.reset_index(inplace=True, drop=True)

X = X[:-1]

In [66]:

train = 0.7
valid = 0.9


x_n = len(X)
y_n = len(y)

X_, y_ = {}, {}

X_['train'] = X[0:int(x_n*train)]
X_['valid'] = X[int(x_n*train):int(x_n*valid)]
X_['test']  = X[int(x_n*valid):]

y_['train'] = y[0:int(y_n*train)]
y_['valid'] = y[int(y_n*train):int(y_n*valid)]
y_['test']  = y[int(y_n*valid):]

num_features = X.shape[1]

num_features

X_

{'train':            AUD       CAD       CHF       EUR       GBP       JPY       NZD  \
 0     0.061881 -0.084664  0.028045 -0.037279  0.040824 -0.010721 -0.147664   
 1    -0.033417  0.125230 -0.050101 -0.012460 -0.073844  0.025457 -0.096032   
 2    -0.032196  0.252279  0.047114  0.011464 -0.026092 -0.043892 -0.052658   
 3     0.052931 -0.112991  0.009011 -0.033590 -0.107902  0.156964  0.005096   
 4     0.020095  0.010240 -0.020462  0.002050 -0.016285 -0.014165  0.046656   
 ...        ...       ...       ...       ...       ...       ...       ...   
 4295 -0.019957  0.013731  0.014446 -0.058569 -0.035316  0.075773 -0.015720   
 4296  0.073803  0.149694 -0.059562  0.012906 -0.010852 -0.246373  0.085951   
 4297  0.114687  0.041749 -0.362267 -0.015757  0.075065 -0.013455 -0.013119   
 4298 -0.153586 -0.053709  0.261702 -0.022577 -0.023860  0.074942 -0.098095   
 4299 -0.071138 -0.031101  0.037765  0.046331  0.003691  0.097222 -0.037097   
 
            USD  
 0     0.149580  
 1   

In [60]:
n_lag = 1
n_neurons = 200
n_epochs = 1000
n_batch = 1
n_neurons = 1

# design network
model = Sequential()

model.add(LSTM(n_neurons, stateful=True))

model.add(Dense(y.shape[1]))

model.compile(optimizer='adam', loss='mse')

In [69]:

MAX_EPOCHS = 10000

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError(), tf.keras.metrics.Accuracy()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  
  return history


In [22]:
compile_and_fit(model, )

In [61]:
model.fit(x=X_['train'], y=y_['train'])

ValueError: in user code:

    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 214, in assert_input_compatibility
        ra